In [ ]:
%matplotlib notebook
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D, art3d

colors = [
    (1, 0, 0),
    (0, 1, 0),
    (0, 0, 1),
    (1, 1, 0),
    (0, 1, 1),
    (1, 0, 1)
]

labels = ['exl', 'enl', 'exr', 'enr', 'sn']

def print_model_code(model_points, precision=2, name=''):
    print(name)
    format_string = (("{:." + str(precision) + "f}, ") * 3)[:-2]
    for label in labels:
        model = format_string.format(*model_points[label])
        print(f'  this->model_points.push_back({"{"}{model}{"}"});  // {label}')
    print()

def load_model(filename):
    model = {}
    with open(filename, 'r') as mf:
        def c(maybe_num):
            try:
                return float(maybe_num)
            except ValueError:
                return maybe_num

        data = [[c(v) for v in l.split(',')] for l in mf.read().splitlines()]
        for l in data:
            model[l[0]] = np.array(l[1:])
    return model

def translate_to_first_point(model):
    for label in labels[::-1]:
        model[label] = model[label] - model[labels[0]]
    return model

def rotate_model(model, rotation, axis='x'):
    for label in labels:
        model[label] = rotate(model[label], rotation, axis)
    return model
    
def rotate(point, rotation, axis='x'):
    angle = rotation / 180 * np.pi
    cos = np.cos(angle)
    sin = np.sin(angle)
    if axis is 'z':
        rot = np.array((
            (cos, -sin, 0),
            (sin, cos, 0),
            (0, 0, 1)
        ))
    elif axis is 'y':
        rot = np.array((
            (cos, 0, sin),
            (0, 1, 0),
            (-sin, 0, cos)
        ))
    else:
        rot = np.array((
            (1, 0, 0),
            (0, cos, -sin),
            (0, sin, cos)
        ))
    return rot @ point

def plot(model, precision, name=''):
    unit_coord = 10 ** -(precision//2 - 1)
    fig = plt.figure(name)
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(195, 40)
    ax.scatter(
        *art3d.rotate_axes(
            *zip(*model.values()), 'y'),
        c=colors[:len(model)]
    )

    ax.plot((0, unit_coord), (0, 0), (0, 0), 'b')
    ax.plot((0, 0), (0, unit_coord), (0, 0), 'r')
    ax.plot((0, 0), (0, 0), (0, unit_coord), 'g')
    plt.show()

In [ ]:
def get_model_0():
    """
    Rough estimates based on 
    https://en.wikipedia.org/wiki/Human_head#/media/File:AvgHeadSizes.png
    """
    return translate_to_first_point({
        'exl': np.array(( 0.0595, -0.0357, -0.0555)),
        'enl': np.array(( 0.013,  -0.0357, -0.0605)),
        'exr': np.array((-0.0595, -0.0357, -0.0555)),
        'enr': np.array((-0.013,  -0.0357, -0.0605)),
        'sn':  np.array(( 0,       0.0435, -0.087))
    }), 4, 'Custom'

In [ ]:
def get_model_1():
    """
    https://www.yobi3d.com/v/4r5otKtbws/HumanHead.stl
    """
    model = load_model('model.csv')
    model = rotate_model(model, 13, 'y')
    model = rotate_model(model, 10, 'z')
    model = rotate_model(model, 90, 'x')
    model = rotate_model(model, -2, 'y')
    model = rotate_model(model, -3, 'z')
    model = translate_to_first_point(model)
    return model, 2, 'Head'

In [ ]:
def get_model_2():
    """
    https://www.myminifactory.com/object/head-lamp-21863
    """
    model = load_model('model2.csv')
    model = rotate_model(model, 30, 'x')
    model = rotate_model(model, 180, 'z')
    model = rotate_model(model, 180, 'y')
    model = translate_to_first_point(model)
    return model, 2, 'head lamp'

In [ ]:
models = [v for k, v in globals().items() if k.startswith('get_model')]

In [ ]:
for model in models:
    print_model_code(*model())

In [ ]:
for model in models:
    plot(*model())